In [1]:
#%%
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential,losses,optimizers,datasets

In [2]:
#层方式实现
x = tf.constant([2.,1.,0.1])
layer = layers.Softmax(axis=-1)
layer(x)

<tf.Tensor: id=2, shape=(3,), dtype=float32, numpy=array([0.6590012, 0.242433 , 0.0985659], dtype=float32)>

In [4]:
layer.__call__(x)

<tf.Tensor: id=4, shape=(3,), dtype=float32, numpy=array([0.6590012, 0.242433 , 0.0985659], dtype=float32)>

In [3]:
# 张量方式实现
tf.nn.softmax(x,axis=-1)

<tf.Tensor: id=3, shape=(3,), dtype=float32, numpy=array([0.6590012, 0.242433 , 0.0985659], dtype=float32)>

In [5]:
from tensorflow.keras import layers, Sequential
# 第一种 构建网络架构方式
network = Sequential([
    layers.Dense(3, activation=None),
    layers.ReLU(),
    layers.Dense(2, activation=None),
    layers.ReLU()
])
x = tf.random.normal([4,3])
network(x)

<tf.Tensor: id=64, shape=(4, 2), dtype=float32, numpy=
array([[0.        , 0.06886908],
       [0.4866067 , 0.        ],
       [0.03505212, 0.        ],
       [0.        , 0.36587328]], dtype=float32)>

In [9]:
layers_num = 2
# 第二种 构建网络架构方式
network = Sequential([])
for _ in range(layers_num):
    network.add(layers.Dense(3))
    network.add(layers.Dropout(0.2))
    network.add(layers.BatchNormalization())
    network.add(layers.ReLU())
network.build(input_shape=(None, 4))
network.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              multiple                  15        
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
batch_normalization (BatchNo multiple                  12        
_________________________________________________________________
re_lu_4 (ReLU)               multiple                  0         
_________________________________________________________________
dense_5 (Dense)              multiple                  12        
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
batch_normalization_1 (Batch multiple                 

In [11]:
for p in network.variables:
    print(p.name, p.shape)

dense_4/kernel:0 (4, 3)
dense_4/bias:0 (3,)
batch_normalization/gamma:0 (3,)
batch_normalization/beta:0 (3,)
batch_normalization/moving_mean:0 (3,)
batch_normalization/moving_variance:0 (3,)
dense_5/kernel:0 (3, 3)
dense_5/bias:0 (3,)
batch_normalization_1/gamma:0 (3,)
batch_normalization_1/beta:0 (3,)
batch_normalization_1/moving_mean:0 (3,)
batch_normalization_1/moving_variance:0 (3,)


In [10]:
for p in network.trainable_variables:
    print(p.name, p.shape)

dense_4/kernel:0 (4, 3)
dense_4/bias:0 (3,)
batch_normalization/gamma:0 (3,)
batch_normalization/beta:0 (3,)
dense_5/kernel:0 (3, 3)
dense_5/bias:0 (3,)
batch_normalization_1/gamma:0 (3,)
batch_normalization_1/beta:0 (3,)


In [12]:
for p in network.non_trainable_variables:
    print(p.name, p.shape)

batch_normalization/moving_mean:0 (3,)
batch_normalization/moving_variance:0 (3,)
batch_normalization_1/moving_mean:0 (3,)
batch_normalization_1/moving_variance:0 (3,)


In [14]:
def proprocess(x,y):
    x = tf.reshape(x, [-1]) 
    return x,y

In [15]:
# x: [60k, 28, 28],
# y: [60k]
(x, y), (x_test,y_test) = datasets.mnist.load_data()
# x: [0~255] => [0~1.]
x = tf.convert_to_tensor(x, dtype=tf.float32) / 255.
y = tf.convert_to_tensor(y, dtype=tf.int32) 

In [16]:
# x: [0~255] => [0~1.]
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32) / 255.
y_test = tf.convert_to_tensor(y_test, dtype=tf.int32) 

In [17]:
train_db = tf.data.Dataset.from_tensor_slices((x,y))
train_db = train_db.shuffle(1000).map(proprocess).batch(128)

val_db = tf.data.Dataset.from_tensor_slices((x_test,y_test))
val_db = val_db.shuffle(1000).map(proprocess).batch(128)

In [18]:
x,y = next(iter(train_db))
print(x.shape, y.shape)

(128, 784) (128,)


In [24]:
# 创建5层的全连接层网络
network = Sequential([layers.Dense(256, activation='relu'),
                     layers.Dense(128, activation='relu'),
                     layers.Dense(64, activation='relu'),
                     layers.Dense(32, activation='relu'),
                     layers.Dense(10)])
network.build(input_shape=(4, 28*28))
network.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             multiple                  200960    
_________________________________________________________________
dense_12 (Dense)             multiple                  32896     
_________________________________________________________________
dense_13 (Dense)             multiple                  8256      
_________________________________________________________________
dense_14 (Dense)             multiple                  2080      
_________________________________________________________________
dense_15 (Dense)             multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


In [25]:
# 导入优化器，损失函数模块
from tensorflow.keras import optimizers,losses 
# 采用Adam优化器，学习率为0.01;采用交叉熵损失函数，包含Softmax
network.compile(optimizer=optimizers.Adam(lr=0.01),
        # 使用 CategoricalCrossentropy 是有问题的 需要对 标签one hot
        loss=losses.SparseCategoricalCrossentropy(from_logits=True),
#         loss=losses.CategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'] # 设置测量指标为准确率
)

In [26]:
# 指定训练集为db，验证集为val_db,训练5个epochs，每2个epoch验证一次
history = network.fit(train_db, epochs=5, validation_data=val_db, validation_freq=2)

Epoch 1/5
469/469 [==============================] - 8s 18ms/step - loss: 0.2758 - accuracy: 0.9185
Epoch 2/5
469/469 [==============================] - 9s 19ms/step - loss: 0.1407 - accuracy: 0.9610 - val_loss: 0.1467 - val_accuracy: 0.9626
Epoch 3/5
469/469 [==============================] - 7s 15ms/step - loss: 0.1069 - accuracy: 0.9702
Epoch 4/5
469/469 [==============================] - 8s 18ms/step - loss: 0.0982 - accuracy: 0.9735 - val_loss: 0.1498 - val_accuracy: 0.9684
Epoch 5/5
469/469 [==============================] - 7s 15ms/step - loss: 0.0848 - accuracy: 0.9779


In [14]:
history.history # 打印训练记录

{'loss': [0.27423731803099316,
  0.14266234541187683,
  0.10474128096103669,
  0.10142248704632123,
  0.08903506728013356],
 'accuracy': [0.9198167, 0.9612167, 0.9713167, 0.97248334, 0.9766],
 'val_loss': [0.15112877672514582, 0.16158023075778274],
 'val_accuracy': [0.9611, 0.9647]}

In [28]:
x,y=next(iter(val_db))
print('predict x:',x.shape)
out=network.predict(x)
print(out.shape)

predict x: (128, 784)
(128, 10)


In [37]:
y.dtype

tf.int32

In [35]:
tf.argmax(out,axis=-1).dtype

tf.int64

In [39]:
out=tf.cast(tf.argmax(out,axis=-1),dtype=tf.int32)

In [47]:
(tf.reduce_sum(tf.cast(y==out,dtype=tf.int32))).numpy()/128.0

0.96875

In [48]:
network.evaluate(val_db)

79/79 [==============================] - 1s 16ms/step - loss: 0.1408 - accuracy: 0.9672


[0.14080193492029852, 0.9672]

##### 第一种 张量保存形式

In [49]:
# 保存模型参数到文件上
network.save_weights('model/weights.ckpt')
print('saved weights.')

saved weights.


In [50]:
del network # 删除网络对象

In [51]:
# 重新创建相同的网络结构
network = Sequential([layers.Dense(256, activation='relu'),
                     layers.Dense(128, activation='relu'),
                     layers.Dense(64, activation='relu'),
                     layers.Dense(32, activation='relu'),
                     layers.Dense(10)])

In [52]:
network.compile(optimizer=optimizers.Adam(lr=0.01),
        loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    ) 

In [53]:
# 从参数文件中读取数据并写入当前网络
network.load_weights('model/weights.ckpt')
print('loaded weights!')

loaded weights!


In [54]:
network.evaluate(val_db)

79/79 [==============================] - 1s 11ms/step - loss: 0.1427 - accuracy: 0.9672


[0.14269267010820819, 0.9672]

##### 第二种保存方式 

In [55]:
network.save("model/model.h5")
print("Saved total model.")

Saved total model.


In [56]:
del network # 删除网络对象

In [57]:
network=tf.keras.models.load_model("model/model.h5")
print("Loaded total model.")

Loaded total model.


In [58]:
network.evaluate(val_db)

79/79 [==============================] - 1s 13ms/step - loss: 0.1406 - accuracy: 0.9672


[0.14062316860499663, 0.9672]

###### 第三种方式

In [59]:
tf.keras.experimental.export_saved_model(network,'model/model-savedmodel')
print("Export saved model.")

Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO

In [60]:
network=tf.keras.experimental.load_from_saved_model('model/model-savedmodel')
print("Load saved model.")

Instructions for updating:
The experimental save and load functions have been  deprecated. Please switch to `tf.keras.models.load_model`.
Load saved model.


In [62]:
network.compile(optimizer=optimizers.Adam(lr=0.01),
        loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    ) 

In [63]:
network.evaluate(val_db)

79/79 [==============================] - 1s 13ms/step - loss: 0.1426 - accuracy: 0.9672


[0.14264785665782947, 0.9672]

## 自定义网络层

In [65]:
class MyDense(layers.Layer):
    def __init__(self,inp_dim,outp_dim):
        super(MyDense,self).__init__()
        self.kernel=self.add_variable('w',[inp_dim,outp_dim],trainable=True)

In [66]:
net=MyDense(4,3)

Instructions for updating:
Please use `layer.add_weight` method instead.


In [67]:
net.variables,net.trainable_variables

([<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
  array([[ 0.12857223, -0.67188877,  0.10854173],
         [-0.7148422 ,  0.35388756, -0.8229322 ],
         [-0.60045105, -0.11635906,  0.01998758],
         [-0.8711252 ,  0.37108064,  0.3993516 ]], dtype=float32)>],
 [<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
  array([[ 0.12857223, -0.67188877,  0.10854173],
         [-0.7148422 ,  0.35388756, -0.8229322 ],
         [-0.60045105, -0.11635906,  0.01998758],
         [-0.8711252 ,  0.37108064,  0.3993516 ]], dtype=float32)>])

In [68]:
class MyDense(layers.Layer):
    def __init__(self,inp_dim,outp_dim):
        super(MyDense,self).__init__()
        self.kernel=self.add_variable('w',[inp_dim,outp_dim],trainable=False)

In [69]:
net=MyDense(4,3)

In [70]:
net.variables,net.trainable_variables

([<tf.Variable 'w:0' shape=(4, 3) dtype=float32, numpy=
  array([[ 0.05058533, -0.11204767, -0.226919  ],
         [-0.48154983,  0.47645557,  0.26176846],
         [ 0.33822715,  0.8563094 ,  0.67875373],
         [ 0.80984914, -0.83430344, -0.40043694]], dtype=float32)>], [])

In [79]:
class MyDense(layers.Layer):
    def __init__(self,inp_dim,outp_dim):
        super(MyDense,self).__init__()
        self.kernel=tf.Variable(tf.random.normal([inp_dim,outp_dim]),trainable=False)

In [80]:
net=MyDense(4,3)

In [81]:
net.variables,net.trainable_variables

([<tf.Variable 'Variable:0' shape=(4, 3) dtype=float32, numpy=
  array([[-0.7111575 ,  0.49573883,  0.5887731 ],
         [ 1.2482699 ,  1.7057481 ,  0.9597773 ],
         [ 1.0286794 ,  0.01827494, -0.0444708 ],
         [ 0.03366759, -0.5966423 ,  0.56632274]], dtype=float32)>], [])

In [82]:
class MyDense(layers.Layer):
    def __init__(self,inp_dim,outp_dim):
        super(MyDense,self).__init__()
        self.kernel=tf.Variable(tf.random.normal([inp_dim,outp_dim]),trainable=True)
    def call(self,inputs,training=None):
        out=inputs@self.kernel
        out=tf.nn.relu(out)
        return out

In [84]:
network=Sequential([
    MyDense(784,256),
    MyDense(256,128),
    MyDense(128,64),
    MyDense(64,32),
    MyDense(32,10),
])
network.build(input_shape=(None,28*28))
network.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_dense_10 (MyDense)        multiple                  200704    
_________________________________________________________________
my_dense_11 (MyDense)        multiple                  32768     
_________________________________________________________________
my_dense_12 (MyDense)        multiple                  8192      
_________________________________________________________________
my_dense_13 (MyDense)        multiple                  2048      
_________________________________________________________________
my_dense_14 (MyDense)        multiple                  320       
Total params: 244,032
Trainable params: 244,032
Non-trainable params: 0
_________________________________________________________________


##### 更普遍方式

In [85]:
class MyModel(keras.Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.fc1=MyDense(28*28,256)
        self.fc2=MyDense(256,128)
        self.fc3=MyDense(128,64)
        self.fc4=MyDense(64,32)
        self.fc5=MyDense(32,10)
    def call(self,inputs,training=None):
        x=self.fc1(inputs)
        x=self.fc2(x)
        x=self.fc3(x)
        x=self.fc4(x)
        x=self.fc5(x)
        return x

In [20]:
# 新建池化层
global_average_layer = layers.GlobalAveragePooling2D()
# 利用上一层的输出作为本层的输入，测试其输出
x = tf.random.normal([4,7,7,2048])
out = global_average_layer(x) # 池化层降维
print(out.shape)

(4, 2048)


In [21]:
# 新建全连接层
fc = layers.Dense(100)
# 利用上一层的输出作为本层的输入，测试其输出
x = tf.random.normal([4,2048])
out = fc(x)
print(out.shape)

(4, 100)


## 预训练

In [22]:
#%%
import  tensorflow as tf
from    tensorflow import keras
from    tensorflow.keras import datasets, layers, optimizers, Sequential, metrics

#%%

In [88]:
#%%
# 加载预训练网络模型，并去掉最后一层
resnet = keras.applications.ResNet50(weights='imagenet',include_top=False)
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [89]:
# 测试网络的输出
x = tf.random.normal([4,224,224,3])
out = resnet(x)
out.shape

TensorShape([4, 7, 7, 2048])

In [90]:
#%%
# 新建池化层
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
# 利用上一层的输出作为本层的输入，测试其输出
x = tf.random.normal([4,7,7,2048])
out = global_average_layer(x)
print(out.shape)

(4, 2048)


In [91]:
#%%
# 新建全连接层
fc = tf.keras.layers.Dense(100)
# 利用上一层的输出作为本层的输入，测试其输出
x = tf.random.normal([4,2048])
out = fc(x)
print(out.shape)

(4, 100)


In [92]:
# 重新包裹成我们的网络模型
mynet = Sequential([resnet, global_average_layer, fc])
mynet.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, None, None, 2048)  23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 100)               204900    
Total params: 23,792,612
Trainable params: 23,739,492
Non-trainable params: 53,120
_________________________________________________________________


In [93]:
#%%
resnet.trainable = False
mynet.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, None, None, 2048)  23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 100)               204900    
Total params: 23,792,612
Trainable params: 204,900
Non-trainable params: 23,587,712
_________________________________________________________________


## 测量工具

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (tf.cast(x_train,tf.float32), y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((tf.cast(x_test,tf.float32), y_test)).batch(32)

In [4]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [7]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [8]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [9]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {:.4f}, Accuracy: {:.4f}, Test Loss: {:.4f}, Test Accuracy: {:.4f}'
    print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

    # Reset the metrics for the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.1333, Accuracy: 96.1283, Test Loss: 0.0674, Test Accuracy: 97.7400
Epoch 2, Loss: 0.0429, Accuracy: 98.6383, Test Loss: 0.0485, Test Accuracy: 98.3700
Epoch 3, Loss: 0.0216, Accuracy: 99.3133, Test Loss: 0.0483, Test Accuracy: 98.4900
Epoch 4, Loss: 0.0131, Accuracy: 99.5800, Test Loss: 0.0561, Test Accuracy: 98.4000
Epoch 5, Loss: 0.0091, Accuracy: 99.6850, Test Loss: 0.0610, Test Accuracy: 98.3600


## 可视化

In [52]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [53]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [54]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (tf.cast(x_train,tf.float32), y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((tf.cast(x_test,tf.float32), y_test)).batch(32)

In [55]:
log_dir=r"log/eight/"
summary_write=tf.summary.create_file_writer(log_dir)

In [56]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

In [57]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [58]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [59]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [60]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [61]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {:.4f}, Accuracy: {:.4f}, Test Loss: {:.4f}, Test Accuracy: {:.4f}'
    
    with summary_write.as_default():
        # 写入标量数据
        tf.summary.scalar("train-loss",float(train_loss.result()),step=epoch)
        tf.summary.scalar("train-acc",float(train_accuracy.result()*100),step=epoch)
        tf.summary.scalar("test-loss",float(test_loss.result()),step=epoch)
        tf.summary.scalar("test-acc",float(test_accuracy.result()*100),step=epoch)
        # 写入图片
        tf.summary.image("val-onebyone-images:",[images[0]],step=epoch)
        # 可是化标签直方图
        tf.summary.histogram("conv1-kernel-hist",model.conv1.kernel,step=epoch)
        tf.summary.histogram("conv1-bias-hist",model.conv1.bias,step=epoch)
        # 查看文本信息
        tf.summary.text('loss-text',str(float(str(tf.argmax(model([images[0]]),axis=-1)[0].numpy()))),step=epoch)
    print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

    # Reset the metrics for the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.1388, Accuracy: 95.8450, Test Loss: 0.0579, Test Accuracy: 98.0500
Epoch 2, Loss: 0.0423, Accuracy: 98.6317, Test Loss: 0.0637, Test Accuracy: 98.0400
Epoch 3, Loss: 0.0236, Accuracy: 99.2550, Test Loss: 0.0611, Test Accuracy: 98.2700
Epoch 4, Loss: 0.0141, Accuracy: 99.5367, Test Loss: 0.0694, Test Accuracy: 98.2400
Epoch 5, Loss: 0.0098, Accuracy: 99.6650, Test Loss: 0.0678, Test Accuracy: 98.4300
